# Windrose plots
This example demonstrates **polar wind visualisations** with earthkit-plots: a **frequency windrose**, a **special windrose** with density blobs, and a stylised **multi-panel hourly windrose** with runway overlays for Kloten Airport.

## Data source and setup
We query the **ECMWF IFS model** via the Open-Meteo API for **10-m wind speed and direction** at Kloten.  
Requests are cached and retried for robustness. Optionally, wind speed can be requested in knots by setting `wind_speed_unit="kn"`.

In [ ]:
import requests_cache
from retry_requests import retry

import openmeteo_requests
from openmeteo_sdk.Variable import Variable

cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

url = "https://ensemble-api.open-meteo.com/v1/ensemble"
params = {
    "latitude": 47.4583,
    "longitude": 8.5480,
    "hourly": ["wind_direction_10m", "wind_speed_10m"],
    "models": "ecmwf_ifs025",
    "forecast_days": 1,
    # "wind_speed_unit": "kn"
}

responses = openmeteo.weather_api(url, params=params, verify=False)
response = responses[0]

# Print coordinates for verification
print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation()} m asl")

## Parsing the API response
The hourly fields for all ensemble members are unpacked into a tidy `pandas.DataFrame`:

- `date`: common hourly time axis (UTC).  
- `wind_direction_10m_member`: direction per member (degrees).  
- `wind_speed_10m_member`: speed per member (m/s by default).

This structure makes it straightforward to select time slices and pass arrays to plotting functions.

In [ ]:
import pandas as pd
import numpy as np

hourly = response.Hourly()
hourly_variables = list(map(lambda i: hourly.Variables(i), range(0, hourly.VariablesLength())))
hourly_wind_direction_10m = filter(lambda x: x.Variable() == Variable.wind_direction and x.Altitude() == 10, hourly_variables)
hourly_wind_speed_10m = filter(lambda x: x.Variable() == Variable.wind_speed and x.Altitude() == 10, hourly_variables)

hourly_data = {"date": pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left"
)}

for variable in hourly_wind_direction_10m:
    member = variable.EnsembleMember()
    hourly_data[f"wind_direction_10m_member{member}"] = variable.ValuesAsNumpy()
for variable in hourly_wind_speed_10m:
    member = variable.EnsembleMember()
    hourly_data[f"wind_speed_10m_member{member}"] = variable.ValuesAsNumpy()

hourly_dataframe = pd.DataFrame(data=hourly_data)

hourly_dataframe.head()

For our windrose plot, we only need the forecast for a single point in time. We'll select the data for the first hour and convert it into two NumPy arrays: one for wind speed and one for wind direction.

In [ ]:
first_hour_data = hourly_dataframe.iloc[14]
wind_speed_api = first_hour_data.filter(like='wind_speed_10m').to_numpy()
wind_direction_api = first_hour_data.filter(like='wind_direction_10m').to_numpy()

print(f"Data prepared for plotting with {wind_speed_api.shape[0]} ensemble members.")

## Frequency Windrose

The **frequency windrose** bins speeds into user-defined **radial bins** and divides direction into **angular sectors**.  

Bars show how often wind falls into each (speed, direction) bin; stacks reveal the speed distribution per direction.

In [ ]:
from earthkit.plots.interactive import Chart, polar

chart = Chart()

# Define the speed bins you want to see
speed_bins = [0, 5, 10, 15, 20, 25]

chart.polar_frequency(
    r=wind_speed_api,
    theta=wind_direction_api,
    radial_bins=speed_bins,
    n_angular_sectors=16
)

chart.title("Polar Frequency of Wind Speed and Direction")

chart.fig.update_layout(polar_barmode='stack')
chart.show(renderer="png")

## Special Windrose

The **special windrose** overlays **kernel-density "blobs"** (von Mises–based) and optional ensemble points to visualise the spread and density.


In [ ]:
chart = Chart()

chart.polar(
    r=wind_speed_api.astype(np.float64),
    theta=wind_direction_api.astype(np.float64),
    show_ensemble_points=True,
    show_density_blobs=True,
    density_method="hybrid_vonmises",
)

chart.title("Wind Speed and Direction")

chart.show(renderer="png")

## Hourly windrose sequence with runways

We build a **6-panel** hourly sequence (UTC) to show short-term evolution.  
Each subplot includes:
- **Density blobs** from the ensemble,  
- An operational **limit circle**,  
- **Runway bearings** for Kloten (10/28, 14/32, 16/34), drawn as radial lines.  

This layout supports quick operational assessment of wind relative to runway orientations.

In [ ]:
import plotly.graph_objects as go

# Special elements
default_klo_runways = [
    {'angle1': 98, 'angle2': 278, 'name': 'KLO 10/28'},
    {'angle1': 137, 'angle2': 317, 'name': 'KLO 14/32'},
    {'angle1': 155, 'angle2': 335, 'name': 'KLO 16/34'}
]
runway_style = {"line": {"color": "rgba(128, 128, 128, 0.7)", "width": 6}}
blob_colors = [
    'rgba(54, 57, 102, 0.8)',
    'rgba(125, 77, 119, 0.8)',
    'rgba(255, 178, 109, 0.6)'
]

# Create the Chart object and main plotting loop
chart = Chart(rows=1, columns=6, specs=[[{"type": "polar"}] * 6])
subplot_titles = [t.strftime('%H:%M (UTC)') for t in hourly_dataframe["date"].iloc[14:20]]
chart._subplot_titles = subplot_titles

for i in range(6):
    row = hourly_dataframe.iloc[i+14]
    wind_speed = row.filter(like='wind_speed_10m').to_numpy()
    wind_direction = row.filter(like='wind_direction_10m').to_numpy()

    # Add windrose density blobs
    traces = polar.windrose(
        r=wind_speed.astype(np.float64),
        theta=wind_direction.astype(np.float64),
        colors=blob_colors,
        show_ensemble_points=False,
        show_density_blobs=True,
        density_method="hybrid_vonmises",
    )
    for trace_list in traces:
        for trace in trace_list:
            chart.add_trace(trace, row=1, col=i + 1)

    # Add limit circle
    chart.add_trace(go.Scatterpolar(
        r=[10] * 361, theta=np.arange(0, 361, 1), mode='lines',
        line=dict(color='red', width=2), showlegend=False
    ), row=1, col=i + 1)

    # Add runway traces
    for runway in default_klo_runways:
        chart.add_trace(go.Scatterpolar(
            r=[0, 8], theta=[runway['angle1'], runway['angle1']], mode='lines',
            showlegend=False, **runway_style
        ), row=1, col=i + 1)
        chart.add_trace(go.Scatterpolar(
            r=[0, 8], theta=[runway['angle2'], runway['angle2']], mode='lines',
            showlegend=False, **runway_style
        ), row=1, col=i + 1)

A custom **dark layout** is applied to every subplot (grid, ticks, ranges, background).

In [ ]:
import copy
from earthkit.plots.interactive.charts import DEFAULT_LAYOUT

# Custom dark layout
custom_polar_settings = copy.deepcopy(DEFAULT_LAYOUT['polar'])

custom_polar_settings['bgcolor'] = '#333333'
custom_polar_settings['angularaxis']['gridcolor'] = '#888888'
custom_polar_settings['angularaxis']['linecolor'] = 'white'
custom_polar_settings['angularaxis']['tickfont'] = {'color': 'white', 'size': 12}
custom_polar_settings['radialaxis']['gridcolor'] = '#888888'
custom_polar_settings['radialaxis']['linecolor'] = 'white'
custom_polar_settings['radialaxis']['tickfont'] = {'color': 'white', 'size': 10}
custom_polar_settings['radialaxis']['range'] = [0, 16]
custom_polar_settings['radialaxis']['angle'] = 45

# Apply to every subplot
for i in range(1, 7):
    polar_key = f"polar{i if i > 1 else ''}"
    chart.fig.update_layout({polar_key: custom_polar_settings})

# Update overall figure
chart._layout_override = {
    'paper_bgcolor': '#333333',
    'font_color': 'white',
    'height': 400,
    'margin': dict(b=20, t=80)
}

In [ ]:
chart.show(renderer="png", width=2000, height=400)